In [ ]:
!pip install tokenizers torch cloud-tpu-client https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl

In [ ]:
!pip install tokenizers torch

In [2]:
import os
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers
import torch as tc
import itertools as it
import torch.nn as nn
from torch.nn import functional as F
import numpy as np

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm

In [3]:
INPUT_NAME = "input.txt"
ENCODING = ["NUM","TOKENIZER"][0]

BLOCK_SIZE = 256
BATCH_SIZE = 64
LEARN_STEPS = 5000
LEARNING_RATE = 3e-4

EVAL_ITERS = 200
EVAL_INTERVAL = 100

DEVICE = 'cuda' if tc.cuda.is_available() else xm.xla_device() if os.environ['COLAB_TPU_ADDR'] else 'cpu'

N_EMBEDDING = 384
N_HEAD = 6
N_LAYER = 6
DROPOUT = 0.2

print(f"{INPUT_NAME} | {ENCODING} | {DEVICE}")

In [4]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(N_EMBEDDING, head_size, bias=False)
        self.query = nn.Linear(N_EMBEDDING, head_size, bias=False)
        self.value = nn.Linear(N_EMBEDDING, head_size, bias=False)
        self.register_buffer('tril', tc.tril(tc.ones(BLOCK_SIZE, BLOCK_SIZE)))

        self.dropout = nn.Dropout(DROPOUT)     

    def forward(self, x):
        _,T,C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)

        wei = self.dropout(wei)

        v = self.value(x)
        return wei @ v

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, N_EMBEDDING)
        self.dropout = nn.Dropout(DROPOUT)

    def forward(self, x):
        out = tc.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(DROPOUT)
        )

        nn.init.xavier_uniform_(self.net[0].weight)
        nn.init.xavier_uniform_(self.net[2].weight)

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()

        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = tc.add(x,self.sa(self.ln1(x)), alpha=1.0)
        x = tc.add(x,self.ffwd(self.ln2(x)), alpha=1.0)
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, N_EMBEDDING)
        self.pos_embedding_table = nn.Embedding(BLOCK_SIZE, N_EMBEDDING)
        self.blocks = nn.Sequential(*[Block(N_EMBEDDING, n_head=N_HEAD) for _ in range(N_LAYER)])
        self.ln_f = nn.LayerNorm(N_EMBEDDING)
        self.lm_head = nn.Linear(N_EMBEDDING, vocab_size)

        #self.register_buffer('pos_embeddings', self.pos_embedding_table(tc.arange(BLOCK_SIZE, device=DEVICE)))

    def forward(self, idx, targets = None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.pos_embedding_table(tc.arange(T, device=DEVICE)) # (T, C)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:      
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -BLOCK_SIZE:]
            logits, _ = self(idx_cond)

            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)

            idx_next = tc.multinomial(probs, num_samples=1)
            idx = tc.cat((idx, idx_next), dim=1)
        return idx

class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, train_loss, validation_loss):
        if (validation_loss - train_loss) > self.min_delta:
            self.counter +=1
            if self.counter >= self.patience:  
                return True
        else:
          self.counter = 0
        return False

In [5]:
with open(INPUT_NAME,"r", encoding="utf-8") as file:
    text = file.read()

if ENCODING == "TOKENIZER":
    vocab_size = 5000

    tokenizer = Tokenizer(models.WordPiece())
    tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
    tokenizer.decoder = decoders.WordPiece()

    trainer = trainers.WordPieceTrainer(vocab_size=vocab_size, special_tokens=["[UNK]"])
    tokenizer.train_from_iterator(text, trainer=trainer)
else:
    chars = sorted(list(set(text)))
    vocab_size = len(chars)


    stoi = { ch:i for i,ch in enumerate(chars) }
    itos = dict(enumerate(chars))

encode = lambda s: [stoi[c] for c in s] if ENCODING == "NUM" else tokenizer.encode(s).ids
decode = lambda l: ''.join([itos[i] for i in l]) if ENCODING == "NUM" else tokenizer.decode(l)

data = tc.tensor(encode(text), dtype = tc.long)

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = tc.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = tc.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = tc.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y

In [6]:
model = BigramLanguageModel()
md = model.to(DEVICE)

@tc.no_grad()
def estimate_loss():
    out = {}
    eval_iters = EVAL_ITERS
    md.eval()

    for split in ['train', 'val']:
        losses = tc.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = md(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    md.train()
    return out

In [7]:
optimizer = tc.optim.AdamW(md.parameters(), lr=LEARNING_RATE, weight_decay=3e-4)
early_stopper = EarlyStopper(patience=3, min_delta=0.1)

for step in range(LEARN_STEPS):
    if step % EVAL_INTERVAL == 0 or step == LEARN_STEPS - 1:
        losses = estimate_loss()
        print(f"[{step}/{LEARN_STEPS}] - Train loss: {losses['train']:.4f}, Val loss: {losses['val']:.4f}")

        if early_stopper.early_stop(losses['train'], losses['val']):
          break

    xb, yb = get_batch('train')

    logits, loss = md(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

[1/10000] - Train loss: 4.9969, Val loss: 4.9794


In [ ]:
test_input = tc.zeros((1,1), dtype=tc.long, device=DEVICE)
print(decode(md.generate(test_input, max_new_tokens=300)[0].tolist()))